In [1]:
import os, sys

import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#from ddgclib._catenoid_clean import *
from ddgclib._curvaturesplay import *
import copy

from timeit import default_timer as timer

In [2]:
def sech(x):
    return 1 / np.cosh(x)

def catenoid(u, v, a):
    x = a * np.cos(u) * np.cosh(v / a)
    y = a * np.sin(u) * np.cosh(v / a)
    z = v
    return x, y, z

In [3]:
def catenoid_clean_N(r, length, refinement=2):

    v_l = -length/2
    v_u = length/2
    a = r

    domain = [
              (0.0, 2 * np.pi),  # u
              (v_l, v_u)  # v
              ]
    HC_plane = Complex(2, domain)
    HC_plane.triangulate()
    for i in range(refinement):
        HC_plane.refine_all()

    # H
    HC = Complex(3, domain)
    bV = set()
    cdist = 1e-8

    u_list = []
    v_list = []
    for v in HC_plane.V:
        #print(f'-')
        #print(f'v.x = {v.x}')
        x, y, z = catenoid(*v.x_a,r)
        #print(f'tuple(x, y, z) = {tuple([x, y, z])}')
        v2 = HC.V[tuple([x, y, z])]

        u_list.append(v.x_a[0])
        v_list.append(v.x_a[1])

        #TODO: Does not work at all:
        boundary_bool = (
                        v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                        #v.x[2] == domain[0][0] or v.x[2] == domain[0][1]
                       # or v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                         )
        if boundary_bool:
            bV.add(v2)

    # Connect neighbours
    for v in HC_plane.V:
        for vn in v.nn:
            v1 = list(HC.V)[v.index]
            v2 = list(HC.V)[vn.index]
            v1.connect(v2)

    # Remerge:
    HC.V.merge_all(cdist=cdist)
    bVc = copy.copy(bV)
    for v in bVc:
        #print(f'bv = {v.x}')
        if not (v in HC.V):
            bV.remove(v)

    H_f = []
    K_f = []

    neck_verts = []
    neck_sols = []
    for vert in HC.V:
        if not (vert in bV):
            for u_i, v_i in zip(u_list, v_list):
                x, y, z = catenoid(u_i, v_i,r)
                va = np.array([x, y, z])
                #print(f'va == vert.x_a = {va == vert.x_a}')
                ba = va == vert.x_a
                if ba.all():
                    #print(f'ba.all() = {ba.all()}')
                    u = u_i
                    v = v_i
                    H_f_i = 0.0
                    K_f_i = -(sech(v/a))**4 / (a**2)
                    H_f.append(H_f_i)
                    K_f.append(K_f_i)
                    #TODO: MERGE NEAR VERTICES

                    if z == 0.0:
                        neck_verts.append(vert.index)
                        neck_sols.append((H_f_i, K_f_i))


    return HC, bV, K_f, H_f, neck_verts, neck_sols

In [4]:
r = 1
length = 2

HC, bV, K_f, H_f, neck_verts, neck_sols = catenoid_clean_N(r, length, refinement=2)

In [5]:
from ddgclib._plotting import *
plot_polyscope(HC)

np.zeros([self.V.size(), self.dim]) = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
self.V[v].index = 0
self.V[v].index = 1
self.V[v].index = 2
self.V[v].index = 3
self.V[v].index = 4
self.V[v].index = 5
self.V[v].index = 6
self.V[v].index = 7
self.V[v].index = 8
self.V[v].index = 9
self.V[v].index = 10
self.V[v].index = 11
self.V[v].index = 12
self.V[v].index = 13
self.V[v].index = 14
self.V[v].index = 15
self.V[v].index = 16
self.V[v].index = 17
self.V[v].index = 18
self.V[v].index = 19
self.V[v].index = 20
self.V[v].index = 21
self.V[v].index = 22
self.V[v].index = 23
self.V[v].index = 24
self.V[v].index = 25
self.V[v].index = 26
self.V[v].index = 27
self.V[v].index = 28
self.V[v].index = 29
self.V[v].index =

In [19]:
number_v = 0
for v in HC.V:
    number_v += 1
    #print(number_v)

boundary_bottom = []
boundary_top    = []

v_l = - length/2
v_u =   length/2

for v in HC.V:
   # print(v.index)
    if v.x[2] == v_l:
        boundary_bottom.append(v)

    if v.x[2] == v_u:
        boundary_top.append(v)

        #print(v.cache[1])

#print(HC.V.cache)

'''
v.index=1
v.x = (1.5430806348152437, -3.779457520505508e-16, 1.0)
'''

'\nv.index=1\nv.x = (1.5430806348152437, -3.779457520505508e-16, 1.0)\n'

In [20]:
boundary_bottom_x = []
for v in boundary_bottom:
    #dummy_item.append(v.index)
    boundary_bottom_x.append(v.x)

boundary_top_x = []
for v in boundary_top:
    #dummy_item.append(v.index)
    boundary_top_x.append(v.x)

In [21]:

#print(boundary_bottom[].index)
boundary_top_x

[(1.5430806348152437, -3.779457520505508e-16, 1.0),
 (-1.5430806348152437, 1.889728760252754e-16, 1.0),
 (9.44864380126377e-17, 1.5430806348152437, 1.0),
 (-2.834593140379131e-16, -1.5430806348152437, 1.0)]

In [22]:
vlist = []
for v in HC.V:
    if (abs(v.x[0]) <= 1e-8) and (0.5 <= v.x[1] <= 2):
        vlist.append(v)

vlist
for v in vlist:
    print(v.x)

(6.904717644627194e-17, 1.1276259652063807, 0.5)
(6.904717644627194e-17, 1.1276259652063807, -0.5)
(6.123233995736766e-17, 1.0, 0.0)
(9.44864380126377e-17, 1.5430806348152437, 1.0)
(9.44864380126377e-17, 1.5430806348152437, -1.0)


In [47]:
print(len(vlist))

AttributeError: 'list' object has no attribute 'x'

In [35]:
def sech(x):
    return 1 / np.cosh(x)

def fun_kappa1(a,v):
    kappa_1 = 1/a *sech(v/a)**2
    return kappa_1

def gaussian_curvature_catenoid(a,v):
    '''
    Analytical Solution for gaussian curvature
    '''
    return -1/a**2 * sech(v/a)**4

def gaussian_curvature(kappa1, kappa2):
    K = kappa1 * kappa2
    return K

def mean_curvature(f0, f1, f2):
    pass

def unit_vector(f1,f2):
    e_ij = np.abs(f2-f1)
    return e_ij




#     N_f0 = v.x_a - np.array([0.0, 0.0, v.x_a[2]]) # Version from Stefan
 #   N_f0 = normalized(N_f0)[0]



In [36]:
kappa = fun_kappa1(r,v.x[2])

In [44]:
vlist[4].x[2]

-1.0

In [39]:
K = gaussian_curvature(kappa, -kappa)
K_c = gaussian_curvature_catenoid(1, -1.0)

print(f'K_c = {K_c}')
print(f'K = {K}')

K_c = -0.17637844761413474
K = -0.17637844761413474


In [28]:
for v in vlist:
    kappa = fun_kappa1(r,v.x[2])
    print(f'kappa = {kappa}')
    print(f'v.x = {v.x}')



kappa = 0.7864477329659276
v.x = (6.904717644627194e-17, 1.1276259652063807, 0.5)
kappa = 0.7864477329659276
v.x = (6.904717644627194e-17, 1.1276259652063807, -0.5)
kappa = 1.0
v.x = (6.123233995736766e-17, 1.0, 0.0)
kappa = 0.41997434161402614
v.x = (9.44864380126377e-17, 1.5430806348152437, 1.0)
kappa = 0.41997434161402614
v.x = (9.44864380126377e-17, 1.5430806348152437, -1.0)
